Search Engine with tools and agents

In [2]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [3]:
## Used the inbuilt tools of wikipedia

api_wrapper_wiki = WikipediaAPIWrapper(top_k_result=1,doc_content_chars_max=1000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [4]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)


arxiv


In [6]:
## CUSTOM TOOLS {RAG TOOL}
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FFE6DFAAD0>, search_kwargs={})

In [12]:
from langchain_core.tools import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any information about Langsmith ")
retriever_tool.name

'langsmith-search'

In [13]:
tools = [wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\87504\\Desktop\\RAGLearning\\.venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=1000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 StructuredTool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x000001FFE6C5D360>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x000001FFE6CC1FC0>)]

## RUN ALL THIS TOOLS WITH AGENTS AND LLM MODELS

In [15]:
## RUN all this tools with Agents and LLM Models

## Tools , LLM -> Agent Executor
import openai
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import os 

groq_api_key = os.getenv("GROQ_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")


llm = ChatGroq(groq_api_key=groq_api_key,model_name='llama-3.1-8b-instant')
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001FFE6CCA470>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FFF96E1BA0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [18]:
## Prompt Template

from langchain_classic import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
from langchain_classic.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [23]:
## Agent Executor

from langchain_classic.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [24]:
agent_executor.invoke({"input" : "Tell Me About Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


LangSmith docs - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewCreate an account and API keyIntegrationsPlansAccount administrationOverviewSet up hierarchyWorkload isolationManage organizations using the APIManage billingGranular usageSet up resource tagsUser managementAdditional resourcesPolly AI assistantBetaData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusLangSmith docsCopy pageCopy pageLangSmith is a framework-agnostic platform for developing, debugging, and deploying AI agents and LLM applications.

​More ways to build
Platform setupUse LangSmith in managed cloud, in a self-hosted environment, or hybrid to match your inf

{'input': 'Tell Me About Langsmith',
 'output': "The Langsmith platform provides several benefits for AI development, including:\n\n1.  **Framework-Agnostic**: Langsmith is a framework-agnostic platform, which means it works with any agent stack, whether you're using an existing framework or building your own.\n2.  **Integrated Workflow**: Langsmith combines observability, evaluation, deployment, and platform setup in one integrated workflow, making it easier to develop, debug, and deploy AI agents.\n3.  **Improved Reliability**: With Langsmith, you can prototype locally and then move to production with integrated monitoring and evaluation, which helps build more reliable AI agents.\n4.  **Data Security and Privacy**: Langsmith meets the highest standards of data security and privacy with HIPAA, SOC 2 Type 2, and GDPR compliance.\n5.  **Collaboration**: Langsmith provides built-in versioning and collaboration, making it easier to iterate on prompts and ship improvements faster.\n6.  **